In [23]:
from pandas_datareader import data as pdr

import yfinance as yf
yf.pdr_override() # <== that's all it takes :-)

# download dataframe
df = pdr.get_data_yahoo("IBM", start="2019-01-01", end="2024-01-01")

[*********************100%%**********************]  1 of 1 completed


In [24]:
df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-01-02,107.084129,110.879539,106.778206,110.143402,85.522606,4434935
2019-01-03,109.493309,109.827919,107.734222,107.944550,83.815262,4546648
2019-01-04,109.856598,112.323135,109.407265,112.160614,87.088905,4683779
2019-01-07,112.332695,113.604210,111.539200,112.954109,87.705017,3923755
2019-01-08,114.397705,115.267685,113.747612,114.560226,88.952118,4982726
...,...,...,...,...,...,...
2023-12-22,161.100006,162.410004,161.000000,162.139999,160.675140,2439800
2023-12-26,162.229996,163.309998,162.050003,163.210007,161.735489,1772400
2023-12-27,163.139999,163.639999,162.679993,163.460007,161.983231,3234600


## Normalize the Data

In [29]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Assuming df is your DataFrame
scaler = StandardScaler()
scaled_features = scaler.fit_transform(df) 

## Sequences

In [30]:
import numpy as np

def create_sequences(data, seq_length):
    xs = []
    ys = []

    for i in range(len(data)-seq_length-1):
        x = data[i:(i+seq_length)]
        y = data[i+seq_length][4]  # Assuming 'Adj Close' is at index 4
        xs.append(x)
        ys.append(y)

    return np.array(xs), np.array(ys)

seq_length = 5  # Example sequence length
X, y = create_sequences(scaled_features, seq_length)

In [44]:
X

array([[[-2.13191001, -1.91135055, -2.02940879, -1.84926603,
         -1.91826948, -0.17143422],
        [-1.91017931, -2.00930576, -1.94263633, -2.05103876,
         -2.02951717, -0.13457174],
        [-1.87674374, -1.77688403, -1.79078332, -1.66416114,
         -1.81621202, -0.08932196],
        [-1.64885425, -1.65755589, -1.59727909, -1.59134786,
         -1.77606721, -0.34011077],
        [-1.45879946, -1.50260831, -1.39683345, -1.44396614,
         -1.69480819,  0.00932303]],

       [[-1.91017931, -2.00930576, -1.94263633, -2.05103876,
         -2.02951717, -0.13457174],
        [-1.87674374, -1.77688403, -1.79078332, -1.66416114,
         -1.81621202, -0.08932196],
        [-1.64885425, -1.65755589, -1.59727909, -1.59134786,
         -1.77606721, -0.34011077],
        [-1.45879946, -1.50260831, -1.39683345, -1.44396614,
         -1.69480819,  0.00932303],
        [-1.34881394, -1.42869596, -1.31960543, -1.3685205 ,
         -1.6532113 , -0.38066632]],

       [[-1.87674374, -1.7

## Split the Data

In [36]:
split_fraction = 0.8
split = int(split_fraction * len(X))

X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

## Convert to Torch Tensor

In [37]:
import torch
from torch.utils.data import TensorDataset, DataLoader

batch_size = 64  # Example batch size

train_data = TensorDataset(torch.Tensor(X_train), torch.Tensor(y_train))
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)

test_data = TensorDataset(torch.Tensor(X_test), torch.Tensor(y_test))
test_loader = DataLoader(test_data, shuffle=False, batch_size=batch_size)

In [45]:
import torch
import torch.nn as nn

class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleRNN, self).__init__()
        self.hidden_size = hidden_size

        # The RNN layer
        self.rnn = nn.RNN(input_size, hidden_size, num_layers=1, batch_first=True)
        
        # Fully connected layer that outputs the final result
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = torch.zeros(1, x.size(0), self.hidden_size)
        
        # Forward propagate the RNN
        out, _ = self.rnn(x, h0)
        
        # Pass the output of the last time step to the classifier
        out = self.fc(out[:, -1, :])
        return out

## Training

In [51]:
import torch.optim as optim

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device == "cuda":
    print("Running on GPU")
else: print("Running on CPU")

# Assuming the SimpleRNN model and its parameters
model = SimpleRNN(input_size=6, hidden_size=20, output_size=1).to(device)
criterion = nn.MSELoss()  # For regression tasks
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 100  # Number of epochs for training
model.train()  # Set model to training mode

for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(inputs)
        
        # Calculate loss
        loss = criterion(outputs, labels.unsqueeze(-1))
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()
        
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluation on test set (this is a very basic form of evaluation)
model.eval()  # Set model to evaluation mode
with torch.no_grad():
    correct = 0
    total = 0
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        
        # Here you would implement your logic to evaluate the model's performance
        # For example, calculating the mean squared error for regression tasks

print('Evaluation done.')


Running on CPU
Epoch [1/100], Loss: 0.3585
Epoch [2/100], Loss: 0.1927
Epoch [3/100], Loss: 0.1369
Epoch [4/100], Loss: 0.1052
Epoch [5/100], Loss: 0.0399
Epoch [6/100], Loss: 0.0293
Epoch [7/100], Loss: 0.0174
Epoch [8/100], Loss: 0.0119
Epoch [9/100], Loss: 0.0183
Epoch [10/100], Loss: 0.0248
Epoch [11/100], Loss: 0.0290
Epoch [12/100], Loss: 0.0357
Epoch [13/100], Loss: 0.0257
Epoch [14/100], Loss: 0.0280
Epoch [15/100], Loss: 0.0154
Epoch [16/100], Loss: 0.0071
Epoch [17/100], Loss: 0.0224
Epoch [18/100], Loss: 0.0208
Epoch [19/100], Loss: 0.0127
Epoch [20/100], Loss: 0.0237
Epoch [21/100], Loss: 0.0154
Epoch [22/100], Loss: 0.0193
Epoch [23/100], Loss: 0.0173
Epoch [24/100], Loss: 0.0265
Epoch [25/100], Loss: 0.0169
Epoch [26/100], Loss: 0.0230
Epoch [27/100], Loss: 0.0235
Epoch [28/100], Loss: 0.0092
Epoch [29/100], Loss: 0.0123
Epoch [30/100], Loss: 0.0083
Epoch [31/100], Loss: 0.0154
Epoch [32/100], Loss: 0.0168
Epoch [33/100], Loss: 0.0166
Epoch [34/100], Loss: 0.0158
Epoch [3

## Regression Eval

In [52]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

model.eval()  # Set the model to evaluation mode

actuals = []
predictions = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)

        actuals.extend(labels.cpu().numpy())
        predictions.extend(outputs.cpu().numpy())

# Convert lists to numpy arrays for evaluation
actuals = np.array(actuals)
predictions = np.array(predictions)

mae = mean_absolute_error(actuals, predictions)
mse = mean_squared_error(actuals, predictions)
rmse = np.sqrt(mse)

print(f'Mean Absolute Error: {mae:.4f}')
print(f'Mean Squared Error: {mse:.4f}')
print(f'Root Mean Squared Error: {rmse:.4f}')


Mean Absolute Error: 0.1354
Mean Squared Error: 0.0460
Root Mean Squared Error: 0.2145
